In [39]:
import json
import time
import requests

In [6]:
prometheus_url = "http://101.202.0.9:30003/api/v1/query"
instance="192.168.10.5:9100"

In [3]:
def prometheus_query(query):
    params = {"query": query}

    response = requests.get(prometheus_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to query Prometheus. Status code: {response.status_code}")
        return None

In [ ]:
query_nw = f"rate(node_network_receive_bytes_total{{instance='{instance}', device!~'^v.*'}}[10s])"
query_ib = f"rate(node_infiniband_port_data_received_bytes_total{{instance='{instance}', device=~'mlx5_[0-9]'}}[10s])"

with open("network.jsonl", "a+") as f:
    for _ in range(10):
        data_nw = prometheus_query(query_nw)
        data_ib = prometheus_query(query_ib)
        json.dump(data_nw, f)
        f.write("\n")
        json.dump(data_ib, f)
        f.write("\n")
        time.sleep(5)